In [41]:
import pytz
import aiohttp
import asyncio
import datetime


async def datetime_to_iso_string(date: datetime.datetime) -> str :
    return (
        date
        .replace(microsecond=0)
        .astimezone(pytz.timezone('Asia/Seoul'))
        .isoformat()
    )


async def request_sampling_data(target_keyword: list, size_per_channel: int, day_interval: int):
    url = 'http://k8s.mysterico.com:31464/analyzer/niz_sample_data/generate'
    async with aiohttp.ClientSession() as session:
        today = datetime.datetime.now()
        iso_today = await datetime_to_iso_string(today)
        interval = datetime.timedelta(days=day_interval)
        response = await session.post(
            url=url,
            json={
                "size_per_channel": size_per_channel,
                "target_keyword": target_keyword,
                "start_date": await datetime_to_iso_string(today - interval),
                "end_date": iso_today,
                "include": ["channel"]
            }
        )
        return await response.json()


async def random_sampling(target_keyword: list, size_per_channel: int, day_interval: int):
    response = await request_sampling_data(target_keyword, size_per_channel, day_interval)
    documents = response.get("documents")
    documents = [document.get("contentPlainText") for document in documents]
    return documents




In [2]:
documents = await random_sampling(
    size_per_channel=1000,
    day_interval=2 ** 10
)
len(documents)dd

6102

In [56]:
import html_text
from unicodedata import normalize as unicode_normalize
# from soynlp.normalizer import emoticon_normalize, repeat_normalize
import re


def validate_doc(doc):
    
    def is_string(doc):
        if isinstance(doc, str): return True
    
    def is_korean(doc):
        first_consonant_ascii_num = 12593 # ㄱ
        last_vowel_ascii_num = 12643 # ㅣ
        first_character_ascii_num = 44032 # 가
        last_character_ascii_num = 55203 # 힣
        for text in doc:
            if first_consonant_ascii_num <= ord(text) <= last_vowel_ascii_num or \
               first_character_ascii_num <= ord(text) <= last_character_ascii_num:
                    return True
    
    if is_string(doc):
        if is_korean(doc):
            return doc
    else:
        return None


class NoneResultError(Exception):
    def __str__(self):
        return '[EMPTY RESULT]'

class PlainTextPreprocessing:

    def __call__(self, text):
        result_str = text
        if not result_str:
            return None
        try:
            result_str = self.extract_plain_text_from_html(result_str)
            # 초성 중성 종성 쪼개진거 합치기
            result_str = unicode_normalize('NFC', result_str)

            #html 태그 제거하다가 특수문자 처리 안된경우
            result_str = self.replace_html_special_char(result_str)
            # delete printed hexa code


            result_str = self.delete_printed_hexa_str(result_str)

            # 한글, 영문, 일부 특수기호 및 감정표현 이모티콘 제외 삭제
            result_str = self.delete_emotion_special_char_duplicate(result_str)
            # print(result_str)
            result_str = self.extract_kor_eng_num_emoji(result_str)
            result_str = self.delete_duplicate_linebreak(result_str)
            result_str = self.delete_duplicate_space(result_str)
            result_str = self.delete_emotion_special_char_duplicate(result_str)
            result_str = result_str.lower()
            result_str = self.delete_fixed_pattern_sns_platform_str(result_str)
            result_str = self.delete_email(result_str)
            result_str = self.delete_url(result_str)
            result_str = self.delete_id_annotation(result_str)

        # 결과가 None이 되는게 아닌 오류 발생시 일단 전처리 PASS 위해
        except Exception as e:
            return None

        return result_str

    @staticmethod
    def delete_printed_hexa_str(in_str):
        result = re.sub(r'\\x[\w]{2}', ' ', in_str)
        return result

    @staticmethod
    def delete_id_annotation(in_str):
        """
        twitter, insta, naver등 id 언급 제거
        @뒤에 숫자, 영문, 한글 연속된 경우 삭제
        """
        result = re.sub(r'@[0-9a-zA-Zㄱ-힣_]{1,51}', '', in_str).strip()

        if not result:
            raise NoneResultError
        return result

    @staticmethod
    def delete_emotion_special_char_duplicate(in_str):

        # result = emoticon_normalize(in_str, num_repeats=2)

        result = repeat_normalize(in_str, num_repeats=2)
        # 한 글자가 3번 이상 연속되는 경우 > 2개로 줄임

        for tmp_repeat in set(re.findall(r'(\S)\1{2,}', result)):
            if not tmp_repeat:
                continue

            regex = r'([' + tmp_repeat + ']){2,}'
            try:
                p = re.compile(regex)
            except re.error as e:
                # 정규표현식용 특수문자 예외처리
                tmp_repeat = f'\\{tmp_repeat}'
                regex = r'([' + tmp_repeat + ']){2,}'
                p = re.compile(regex)

            result = p.sub(r'\1\1', result)
        # print(result)
        # 1~4 글자가 동일하게 반복되는 경우 1개로 삭제
        regex = r'(\S)([\S\W])?([\S\W])?([\S\W])?(\1{1}\2?\3?\4?){3,}'
        p = re.compile(regex)

        result = p.sub(r'\1\2\3\4', result)

        if not result:
            raise NoneResultError

        return result

    @staticmethod
    def extract_kor_eng_num_emoji(in_str):
        """
            남기기
            - \u0020-\u007E : 기본 ASCII 범위 영문자, 문장부호, 숫자 포함
            - \u2010-\u2064 : 추가 수식
            - \u00a9 : copyright
            - \u2600-\u26FF : 전화기 표시, 우산 등 빈출 이모티콘
            - ㄱ-ㅣ가-힣
            - \u1F300-\u1F64F 감정표현 및
            - \u1F900-\u1F9FF 추가 이모티콘

            +++비슷한 감정 축약
            ~~~ U+1F487 까지만 쓰자
        """
        mod_str = re.sub(
            r'[^\u0020-\u007Eㄱ-ㅣ가-힣\u2010-\u2064\u00a9\u2600-\u26FF\U0001F300-\U0001F64F\U0001F900-\U0001F9FF]',
            ' ', in_str).strip()

        # 웃는표정들 하나로 축약
        mod_str = re.sub(
            r'[\U0001F600-\U0001F606\U0001F619-\U0001F61D\U0001F609-\U0001F60E\U0001F638-\U0001F63D\U0001F642\U0001F645-\U0001F647\U0001F64B-\U0001F64C]',
            '\U0001F600', mod_str).strip()

        if not mod_str:
            raise NoneResultError

        return mod_str

    @staticmethod
    def extract_plain_text_from_html(in_str):
        result = html_text.extract_text(in_str)
        if not result:
            raise NoneResultError
        return result

    @staticmethod
    def delete_email(in_str):
        return re.sub(r'[a-zA-Z0-9-\.]+@([a-zA-Z0-9-]+.)+[a-zA-Z0-9-]{2,4}', '메일', in_str)

    @staticmethod
    def delete_url(in_str):
        def pass_exception_case(in_case):
            # 소수점에 대해서 처리X ex) 30.1

            try:

                if not in_case[0]:
                    return in_case[0]

                if in_case[0].find('http') > -1 and in_case[0].find('//') > -1:
                    # print(f'{in_case[0]} > 링크 pass')
                    return ''

                # 짧은 문자열에 대한 처리
                # 1. 짧은 도메인 규칙 + 영문만으로 된 도메인 룰이면서 짧은거 링크로 간주
                if re.fullmatch(r'(([a-zA-Z\-]+\.)+[a-zA-Z\-]{2,5})', in_case[0]):
                    fullmatched_text = re.fullmatch(r'(([a-zA-Z\-]+\.)+[a-zA-Z\-]{2,5})', in_case[0])
                    if len(fullmatched_text[0]) < 7:
                        return ''
                # 도메인 룰에 어긋나지만 너무 짧은건 패스
                elif len(in_case[0]) < 7:
                    # print(f'{in_case[0]} > 짧아 pass')
                    return in_case[0]

                # 정규표현식으로 찾은 표현에 숫자가 글자보다 많은 경우 숫자 소수점 표현으로 간주
                if len(re.sub(r'[^\d\-\~\%\.]', '', in_case[0])) / len(in_case[0]) >= 0.5:
                    # print(f'{in_case[0]} > 숫자 pass')
                    return in_case[0]
                # print(f'{in_case[0]} > 링크')
            except Exception as e:
                print(e)
                print(in_case)

            return ''

        regex = re.compile(
            r'(http[s]?:\/\/((www\.)|(m\.))?|ftp:\/\/(www\.)?|www\.)?([0-9a-zA-Z\%\_\-]+\.)+[0-9a-zA-Z\%\_\-]{2,5}(\/[0-9a-zA-Z\%\_\-]+)*(\.[0-9a-zA-Z\%\_\-]+)?\/?(([?#](([\w%]+)?=[\w\%\+\:\-\.]*[&]?(%26)?)+)|[\?]|([\/a-zA-Z0-9_\-]?)*)*')

        return regex.sub(pass_exception_case, in_str)

    @staticmethod
    def delete_duplicate_linebreak(in_str):
        try:
            in_str = in_str.replace('\\n', ' ')
            if in_str:
                in_str = in_str.replace('\\r', ' ')
        except:
            result = in_str

        result = re.sub(r'[\n\r]{1,}', ' ', in_str)

        if not result:
            raise NoneResultError

        return result

    @staticmethod
    def delete_duplicate_space(in_str):
        result = re.sub(r'[\s\u200b\u200c\ufeff]{2,}', ' ', in_str)
        if not result:
            raise NoneResultError

        return result

    @staticmethod
    def delete_fixed_pattern_sns_platform_str(in_str):
        """
        플랫폼마다 반복되는 문자 제거
        """

        """
        반복 문자 제거
        """
        fixed_str_list = ['dc official app', '[eng]', '[sub]', '비디오를 보시려면 원문링크를 클릭하여 확인하세요.']

        in_str = in_str.lower()
        for fixed_str in fixed_str_list:
            in_str = in_str.replace(fixed_str, ' ')
            if not in_str:
                raise NoneResultError

        """
        반복 패턴 제거
        """
        pattern_list = [
            {
                # 네이버 뉴스
                'pattern': r'[\.\n\r][\S\s]{,5}기자\s메일[\S\s]*무단\s전재\s및\s재배포\s금지',
                'replace': r'.'
            },
            {
                # 네이버 뉴스
                'pattern': r'\.*[\sa-zA-Z0-9ㄱ-힣]*기자[\S\s]*무단\s전재\-재배포\s금지',
                'replace': r'.'
            },
            {
                # 네이버 카페 컨텐츠
                'pattern': r'\[{1,3}content\-element\-\d*\]{1,3}',
                'replace': r' '
            }
        ]
        for tmp_pattern in pattern_list:
            sub_tmp_pattern = re.compile(tmp_pattern['pattern'])
            in_str = sub_tmp_pattern.sub(tmp_pattern['replace'], in_str)

        return in_str

    @staticmethod
    def replace_html_special_char(in_str):
        case_list = [
            {
                'html': '&gt;',
                'text': '>'
            },
            {
                'html': '&lt;',
                'text': '<'
            },
            {
                'html': '&amp;',
                'text': '&'
            },
            {
                'html': '&quot;',
                'text': '"'
            },
            {
                'html': '&#039;',
                'text': "'"
            },
            {
                'html': '&nbsp;',
                'text': " "
            },
            {
                'html': '"',
                'text': '"'
            },
            {
                'html': '\'',
                'text': "'"
            }
        ]
        result = in_str
        for tmp_case in case_list:
            result = result.replace(tmp_case['html'], tmp_case['text'])

        return result

In [3]:
import unicodedata
from typing import List

async def request_ner(sentences: List[str]):
    url = 'http://127.0.0.1:8000/predict'
    sentences = [unicodedata.normalize('NFC', sentence) for sentence in sentences]
    async with aiohttp.ClientSession() as session:
        response = await session.post(
            url=url,
            json={
                "sentences": sentences
            }
        )
        return await response.json()

In [4]:
await request_ner(['🐛 Update : 쿠버플로우 auth 버그 캐치를 위한 커스텀 스크립트 추가'])

[{'sentence': '🐛 Update : 쿠버플로우 auth 버그 캐치를 위한 커스텀 스크립트 추가',
  'entities': [['쿠버플로우', 'PS'],
   ['auth', 'PS'],
   ['커스텀', 'PS'],
   ['스크립트', 'PS']]}]

In [5]:
import random

random.choice(documents)

"동생이 경양식 돈까쓰 땡긴다고 퇴근하고 같이 먹으러 갔다 \u200b 나빼고 다(가족들) 가본 돈까스브로스 웃긴건 난 한번도 안 가봤는데 왜 다 내 번호로 적립하는데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 가끔씩 적립되었다고 카톡이 온다 (누군데...누가먹었는데.........?...) \u200b 드디어 나도 왔다 돈까스브로스 철산점 경기도 광명시 철산로 15 2-2호 스프가 나와주구요 동생이 떠다준 궁물 후루룩 다 먹으니까 또 채워주신 숲 경양식 돈까스만의 매력이잖아요 브로스돈까스와 매콤 돈까스 진짜 맛있었다.....조합인정 배 안 찰것 같았는데 다 먹고 배 엄청 불렀다는거^^...; \u200b 너무 배불러서 그렇게 쇼핑을 했습니다 JAJU도 들리고 올리브영도 들리고 (잠시후 올영에서 뭐샀는지 공개) 점심 안 먹으려고 집에 있던 호빵을 챙겨왔다 모짜렐라치즈피자 먹어보셨어요? \u200b 이거 진짜 대박 맛있어요 저의 세젤 호빵이 될 듯 합니다 몇 개 사서 쟁여두ㅓ야지 \u200b 퇴근하고 집가는데 닭볶음탕이 너무 먹고싶었다 배민 넘 많이 시켜먹어서 오랜만에 요리를 해먹자는 다짐을 ㅋㅋㅋㅋㅋㅋㅋ🧡 절단육+버섯+감자=8500원의 행복 \u200b 젤 자신있는 요리 닭볶음탕을 만들어보았습니다 보글보글 버섯 넘 귀요미들인데 버섯전골 같은 비쥬얼 파 고추 송송 썰어 졸여주면 됩니다 (궁물이 없어질때까지) \u200b 유튭을 틀어놓고 밥을 먹겠습니다 요즘 세상 젤 재밌는 곽튜브 곽준빈님 볼때마다 피식.......피쉭... 매콤한 닭볶음탕에 치즈이불을 덮고 와인이랑 먹습니다 이것이 진정한 꼬꼬뱅 아닌가요 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 너모 맛있게 잘 되서 두그릇이나 먹었답니다 \u200b 오랜만에 요리하니까 더 즐겁습니다 나름 거금을 들여산 엡손PM401포프는 간간히 잘 쓰이고 있다 얼마전에 만든 일러스트 작업물이 선명하게 잘 나와준다 이웃 블로그보다가 1월에 삼쩜삼 해서 환급받았는데 설마 또 나오겠어? 했다가 정말로 나왔다 수수료 4000원 내고 바로 환급금 신청 \u200b 애드포스트도 이젠 환

In [54]:
import time
import random

characters_per_line = 40
sampling_trial = 20
target_keyword = [
    "삼쩜삼"
]

documents = await random_sampling(
    target_keyword=target_keyword,
    size_per_channel=1000,
    day_interval=2 ** 10
)
random_documents = [random.choice(documents) for _ in range(sampling_trial)]

print(f'total documents : {len(documents)}')
print(f'sampled documents : {len(random_documents)}\n')

for random_document in random_documents:
    try:
        start = time.time()
        result = await request_ner([random_document])
        for k in result[0]:
            # if k != 'merged_sentence':
                # continue
            print(f'<<{k}>>')
            result = result[0].get(k)
            if k == 'merged_sentence':
                lines = [
                    result[i:i+characters_per_line] 
                    for i in range(
                        0, len(result)-characters_per_line, characters_per_line
                    )
                ]
                result = '\n'.join(lines)
                print(result, '\n')

            else:
                print(result, '\n')
        print(time.time() - start, 'sec')
        print('='*100, '\n')
    except Exception as e:
        print('\n', e)
        print(random_document, '\n')
        continue

total documents : 6102
sampled documents : 20

<<sentence>>
삼쩜삼 시벌 들어왔어???!!!!!!!!!!! 거의 3~4주만이야 미춌다으으ㅡ으ㅡ 더 히바 럭드 공지내놔 럭드 갈겨 https://t.co/Nbr1o7gCTf 

<<merged_sentence>>

 'str' object has no attribute 'get'
삼쩜삼 시벌 들어왔어???!!!!!!!!!!! 거의 3~4주만이야 미춌다으으ㅡ으ㅡ 더 히바 럭드 공지내놔 럭드 갈겨 https://t.co/Nbr1o7gCTf 

<<sentence>>
👀명품기업에 숨어든 가짜직원을 찾아라! 갓챠!🎯

강소기업과 유니콘기업 등 우리 주위에
숨은 명품중소기업을 찾아 소개한다!

제 2편 : 자비스앤빌런즈

‘You work, We help’
복잡하고 번거로운 세금환급은 자비스앤빌런즈가 책임진다!
출시 2년 만에 1000만 고객을 확보한 
삼쩜삼을 개발한 텍스테크 기업, 
자비스앤빌런즈에 가짜 직원이 숨어 있다고?

유망한 기업에 가짜직원이라니
가짜를 찾으러 출동!🕵🏻‍♀️

-----------------------------------------------------------------------------

🔥2편 "자비스앤빌런즈 편" 댓글이벤트🔥

세금환금 쩜 쉽게 도와주는 자비스앤빌런즈를 응원하고
치킨 받아가세요~🧚🏼‍♂

⭐참여방법 : 
1. 갓챠 "자비스앤빌런즈 편"을 시청하고 
    https://youtu.be/bmSUQWOQhjM
2. 영상 댓글에 명품기업 "자비스앤빌런즈" 응원하기!
3. 이벤트 추첨을 위한 폼까지 작성하면 마무리~
    https://naver.me/G1FLk7HZ
 
⭐이벤트기간 : 10월7일(금) ~ 10월14일(금)
⭐당첨자발표 : 10월17일(화)
⭐이벤트경품 : 치킨 (10명)
⭐당첨자 게시 : 중기부 유튜브 커뮤니티 기재
⭐추첨방법 : 영상 댓글 작성과 네이버 폼 작성을 완료한 신청자 한에서 추첨
       